# LSTM

In [ ]:
import os
import pandas as pd
import numpy as np
import re
import math
import shutil
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
import csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Masking
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
import sys
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load a single file as a numpy array
import pandas as pd
import numpy as np
import re

def load_file(filepath):
  tmpList= []
  tmpList1 = []
  tmpList2 = []
  with open(filepath,'r') as thecsv:
    for line in thecsv:
      line = re.sub(re.compile(r'\s+'), '', line)
      line = line.strip(",")
      tmpList.append(line.split(',')[0:24])

    for i in range(len(tmpList)):
      tmpList1 = []
      for element in tmpList[i]:
        tmpList1.append((float(element)))
      tmpList2.append(tmpList1)
  dataframe2 = pd.DataFrame(tmpList2)
  # dataframe2.to_csv('./drive/My Drive/Ashwini/gestureData/test1/dataframe3.csv')
  dataframe2 = dataframe2.replace(np.nan,0.0, regex=True)
  # dataframe2.to_csv('./drive/My Drive/Ashwini/gestureData/test1/dataframe3with0.csv')
  dataframe = dataframe2.astype(float)
  return dataframe.values

def load_file_y(filepath):
  dataframe_y = read_csv(filepath, header=None)
  file = open(filepath, "r")
  csv_reader = csv. reader(file)
  lists_from_csv = []
  for row in csv_reader:
    lists_from_csv. append(int(row[0])-1)
  # return dataframe_y.values.tolist()
  return lists_from_csv

In [ ]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
  loaded = list()
  for name in filenames:
    data = load_file(prefix + name)
    loaded.append(data) 
  # stack group so that features are the 3rd dimension
  loaded = dstack(loaded)
  print("(Total samples, Timesteps, features) - ", loaded.shape)
  return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
  filepath = prefix + "/"
  filenames = list()
  # filenames += ['leftshoulder_velocity.txt','leftshoulder_acceleration.txt','leftelbow_velocity.txt','leftelbow_acceleration.txt','leftwrist_velocity.txt', 'leftwrist_acceleration.txt',]
  # filenames += ['rightshoulder_velocity.txt','rightshoulder_acceleration.txt','rightelbow_velocity.txt','rightelbow_acceleration.txt','rightwrist_velocity.txt', 'rightwrist_acceleration.txt',]
  filenames += ['leftshoulder_polarangle.txt','leftshoulder_velocity.txt','leftelbow_polarangle.txt','leftelbow_velocity.txt','leftwrist_polarangle.txt', 'leftwrist_velocity.txt']
  filenames += ['rightshoulder_polarangle.txt','rightshoulder_velocity.txt','rightelbow_polarangle.txt','rightelbow_velocity.txt','rightwrist_polarangle.txt', 'rightwrist_velocity.txt']
  # load input data
  X = load_group(filenames, filepath)
  # load class output
  y = load_file_y( './drive/My Drive/CS298/2D/' + '/y.txt')
  return X, y

In [ ]:

from sklearn.model_selection import train_test_split



# load the dataset, returns train and test X and y elements
def load_dataset(prefix='./drive/My Drive/CS298/2D/polar_angular_6joints'):
  # load all train
  # trainX, trainy = load_dataset_group('train', prefix)
  # load all test
  X, y = load_dataset_group('doesntmatter', prefix)
  trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.25, random_state=42)
  # zero-offset class values
  # trainy = trainy - 1
  # testy = testy - 1
  # one hot encode y
  trainy = to_categorical(trainy)
  testy = to_categorical(testy)

  return trainX, trainy, testX, testy

In [ ]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  verbose, epochs, batch_size = 0, 20, 117
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  print("Timesteps - ",n_timesteps, "Features - ",n_features, "Output Dimension - ",n_outputs)
  model = tf.keras.models.Sequential()
  model.add(Dropout(0.5))
  model.add(tf.keras.layers.Masking(mask_value=0.0,input_shape=(n_timesteps, n_features)))
  model.add(tf.keras.layers.LSTM(117)) #######need to add the code to directly pick sample -> now it is manually written as 117
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # Confusion matrix
  predictions = model.predict(testX)
  # matrix = confusion_matrix(testy.argmax(axis=1), predictions.argmax(axis=1))
  # print(matrix)
  print(classification_report(testy.argmax(axis=1), predictions.argmax(axis=1)))
  # evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=2)
  # _, accuracy = model.evaluate(testX, testy, verbose=0)
  # accuracy = 0
  return accuracy

In [ ]:
# summarize scores
def summarize_results(scores):
  print(scores)
  m, s = mean(scores), std(scores)
  print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
# run an experiment
def run_experiment(repeats=3):
  # load data

  trainX, trainy, testX, testy = load_dataset()
  scores = list()
  for r in range(repeats):
    print("\n")
    print("Running LSTM with Polar angle and Angular velocity - 2 gestures" )
    score = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)

In [ ]:
run_experiment()


(Total samples, Timesteps, features) -  (117, 24, 12)


Running LSTM with Polar angle and Angular velocity - 2 gestures
Timesteps -  24 Features -  12 Output Dimension -  2
              precision    recall  f1-score   support

           0       0.75      0.71      0.73        17
           1       0.64      0.69      0.67        13

    accuracy                           0.70        30
   macro avg       0.70      0.70      0.70        30
weighted avg       0.70      0.70      0.70        30

1/1 - 2s - loss: 0.6303 - accuracy: 0.7000 - 2s/epoch - 2s/step
>#1: 70.000


Running LSTM with Polar angle and Angular velocity - 2 gestures
Timesteps -  24 Features -  12 Output Dimension -  2
              precision    recall  f1-score   support

           0       0.83      0.29      0.43        17
           1       0.50      0.92      0.65        13

    accuracy                           0.57        30
   macro avg       0.67      0.61      0.54        30
weighted avg       0.69      0.57

# CNN and LSTM

In [ ]:
import os
import pandas as pd
import numpy as np
import re
import math
import shutil
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
import csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Masking
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
import sys
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load a single file as a numpy array
import pandas as pd
import numpy as np
import re

def load_file(filepath):
  tmpList= []
  tmpList1 = []
  tmpList2 = []
  with open(filepath,'r') as thecsv:
    for line in thecsv:
      line = re.sub(re.compile(r'\s+'), '', line)
      line = line.strip(",")
      tmpList.append(line.split(','))

    for i in range(len(tmpList)):
      tmpList1 = []
      for element in tmpList[i]:
        tmpList1.append((float(element)))
      tmpList2.append(tmpList1)
  dataframe2 = pd.DataFrame(tmpList2)
  # dataframe2.to_csv('./drive/My Drive/Ashwini/gestureData/test1/dataframe3.csv')
  dataframe2 = dataframe2.replace(np.nan,0.0, regex=True)
  # dataframe2.to_csv('./drive/My Drive/Ashwini/gestureData/test1/dataframe3with0.csv')
  dataframe = dataframe2.astype(float)
  return dataframe.values

def load_file_y(filepath):
  dataframe_y = read_csv(filepath, header=None)
  return dataframe_y.values

In [ ]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
  loaded = list()
  for name in filenames:
    data = load_file(prefix + name)
    loaded.append(data) 
  # stack group so that features are the 3rd dimension
  loaded = dstack(loaded)
  print("(Total samples, Timesteps, features) - ", loaded.shape)
  return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
  filepath = prefix + "/"
  filenames = list()
  # filenames += ['leftshoulder_velocity.txt','leftshoulder_acceleration.txt','leftelbow_velocity.txt','leftelbow_acceleration.txt','leftwrist_velocity.txt', 'leftwrist_acceleration.txt',]
  # filenames += ['rightshoulder_velocity.txt','rightshoulder_acceleration.txt','rightelbow_velocity.txt','rightelbow_acceleration.txt','rightwrist_velocity.txt', 'rightwrist_acceleration.txt',]
  filenames += ['leftshoulder_polarangle.txt','leftshoulder_velocity.txt','leftelbow_polarangle.txt','leftelbow_velocity.txt','leftwrist_polarangle.txt', 'leftwrist_velocity.txt']
  filenames += ['rightshoulder_polarangle.txt','rightshoulder_velocity.txt','rightelbow_polarangle.txt','rightelbow_velocity.txt','rightwrist_polarangle.txt', 'rightwrist_velocity.txt']
  # load input data
  X = load_group(filenames, filepath)
  # load class output
  y = load_file_y( './drive/My Drive/CS298/' + '/y.txt')
  return X, y

In [ ]:

from sklearn.model_selection import train_test_split



# load the dataset, returns train and test X and y elements
def load_dataset(prefix='./drive/My Drive/CS298/polar_angular_6joints'):
  # load all train
  # trainX, trainy = load_dataset_group('train', prefix)
  # load all test
  X, y = load_dataset_group('doesntmatter', prefix)
  trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.25, random_state=42)
  # zero-offset class values
  trainy = trainy - 1
  testy = testy - 1
  # one hot encode y
  #print("Before_Categorization",trainy.shape,testy.shape)
  trainy = to_categorical(trainy)
  testy = to_categorical(testy)
  #print("After_Categorization",trainy.shape,testy.shape)
  return trainX, trainy, testX, testy

In [ ]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  verbose, epochs, batch_size = 0, 30, 64
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  print("Timesteps - ",n_timesteps, "Features - ",n_features, "Output Dimension - ",n_outputs)
  model = tf.keras.models.Sequential()
  model.add(Dropout(0.5))
  model.add(tf.keras.layers.Masking(mask_value=0.0,input_shape=(n_timesteps, n_features)))
  model.add(tf.keras.layers.LSTM(117)) #######need to add the code to directly pick sample -> now it is manually written as 117
  model.add(Dense(100, activation='tanh'))
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # Confusion matrix
  predictions = model.predict(testX)
  # matrix = confusion_matrix(testy.argmax(axis=1), predictions.argmax(axis=1))
  # print(matrix)
  print(classification_report(testy.argmax(axis=1), predictions.argmax(axis=1)))
  # evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=2)
  # _, accuracy = model.evaluate(testX, testy, verbose=0)
  # accuracy = 0
  return accuracy

In [ ]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	# define model
	verbose, epochs, batch_size = 0, 25, 60
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	# reshape data into time steps of sub-sequences
	n_steps, n_length = 4,10
	trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
	testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
	# define model
	model = Sequential()
	model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
	model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
	model.add(TimeDistributed(Dropout(0.5)))
	model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
	model.add(TimeDistributed(Flatten()))
	model.add(LSTM(100))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

In [ ]:
# summarize scores
def summarize_results(scores):
  print(scores)
  m, s = mean(scores), std(scores)
  print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
# run an experiment
def run_experiment(repeats=3):
  # load data

  trainX, trainy, testX, testy = load_dataset()
  scores = list()
  for r in range(repeats):
    print("\n")
    print("Running LSTM with Polar angle and Angular velocity - 2 gestures" )
    score = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)

In [ ]:
run_experiment()


(Total samples, Timesteps, features) -  (117, 43, 12)


Running LSTM with Polar angle and Angular velocity - 2 gestures


ValueError: ignored